In [6]:
import gseapy as gp
import pandas as pd

slm_genes = [
    "Reln","Gad1","Gad2","Ociad2","Gabra5","Gabra2","Dcn","Lama2","Aqp4",
    "Kcnk2","Kcnc1","Grin2a","Grin2b","Dlg2","Camk2a","Syt1","Bdnf","Slc17a7","Snap25"
]

enr = gp.enrichr(
    gene_list = slm_genes,
    organism  = "Mouse",  # 若是人类基因，改成 "Human"
    gene_sets = [
        "GO_Biological_Process_2023",
        "GO_Cellular_Component_2023",
        "GO_Molecular_Function_2023",
    ],
    outdir = "enrichr_out",
    cutoff = 1.0  # 这里的 cutoff 只影响自动保存的表图；我们下面手动筛选
)

# 1) 拿到总结果表
df = enr.results.copy()         # 对于 enrichr，用 .results
# 若你用的是 GSEA 类方法才常用 .res2d

print("All results rows:", len(df))
print(df.head())

# 2) 选某个本体子集（比如 BP）
df_bp = df[df["Gene_set"]=="GO_Biological_Process_2023"].copy()

# 3) 如果严格显著为空，放宽阈值；若仍为空，就直接取最小的前10个
sig = df_bp[df_bp["Adjusted P-value"] <= 0.05].copy()
if sig.empty:
    # 放宽阈值（比如 0.25），如果还没有，就取 p 调整值最小的前10个
    relaxed = df_bp[df_bp["Adjusted P-value"] <= 0.25].copy()
    plot_df = (relaxed if not relaxed.empty else
               df_bp.sort_values("Adjusted P-value").head(10))
    # 为避免 gseapy 在内部再次用 cutoff 过滤，这里把 cutoff 设大一点
    ax = gp.barplot(plot_df, column="Adjusted P-value",
                    title="GO BP (top terms; relaxed threshold)",
                    ofname="enrichr_out/go_bp_top10.png",
                    cutoff=1.0)
else:
    # 有显著项，画前10个
    plot_df = sig.sort_values("Adjusted P-value").head(10)
    ax = gp.barplot(plot_df, column="Adjusted P-value",
                    title="GO BP (FDR ≤ 0.05)",
                    ofname="enrichr_out/go_bp_top10.png",
                    cutoff=1.0)

print("Plotted rows:", len(plot_df))


All results rows: 402
                     Gene_set  \
0  GO_Biological_Process_2023   
1  GO_Biological_Process_2023   
2  GO_Biological_Process_2023   
3  GO_Biological_Process_2023   
4  GO_Biological_Process_2023   

                                                Term Overlap       P-value  \
0        Chemical Synaptic Transmission (GO:0007268)  10/273  1.578372e-14   
1  Anterograde Trans-Synaptic Signaling (GO:0098916)   7/199  3.955072e-10   
2  Regulation Of Postsynaptic Membrane Potential ...    4/35  2.988537e-08   
3            Neurotransmitter Transport (GO:0006836)    4/52  1.529550e-07   
4  Regulation Of Synaptic Transmission, Glutamate...    4/55  1.923439e-07   

   Adjusted P-value  Old P-value  Old Adjusted P-value  Odds Ratio  \
0      4.482578e-12            0                     0   83.303760   
1      5.616203e-08            0                     0   60.122830   
2      2.829148e-06            0                     0  171.612903   
3      1.085980e-05           

In [7]:
import pandas as pd
df = enr.results.copy()
bp = df[df["Gene_set"]=="GO_Biological_Process_2023"]
top = bp.sort_values("Adjusted P-value").head(10)
print(top[["Term","Adjusted P-value","Overlap","Genes"]])


                                                Term  Adjusted P-value  \
0        Chemical Synaptic Transmission (GO:0007268)      4.482578e-12   
1  Anterograde Trans-Synaptic Signaling (GO:0098916)      5.616203e-08   
2  Regulation Of Postsynaptic Membrane Potential ...      2.829148e-06   
3            Neurotransmitter Transport (GO:0006836)      1.085980e-05   
4  Regulation Of Synaptic Transmission, Glutamate...      1.092513e-05   
5  Positive Regulation Of Synaptic Transmission (...      3.565118e-05   
6  Positive Regulation Of Synaptic Transmission, ...      3.928321e-05   
7  Positive Regulation Of Excitatory Postsynaptic...      3.928321e-05   
8  Synaptic Transmission, Glutamatergic (GO:0035249)      6.612408e-05   
9  Modulation Of Excitatory Postsynaptic Potentia...      9.123584e-05   

  Overlap                                              Genes  
0  10/273  GABRA2;SNAP25;GRIN2A;DLG2;SYT1;GABRA5;GAD1;GAD...  
1   7/199           SNAP25;GRIN2A;DLG2;SYT1;GAD1;GAD2;GRIN2